# Quiz 6 - Object Recognition: BoF vs ConvNets
#### Edson Roteia Araujo Junior e João Pedro Moreira Ferreira

### Instructions
The goal of this quiz is to implement two object recognition approaches:

1.     A classifier based on bag of features:

    - Use SVM or Random Forest as classifiers.

    - Try different sizes for the dictionary.

2.     A classifier using ConvNet implemented in Keras:
 
     - Use an architecture inspired in the LeNet5

Your code must be implemented on a notebook python and you must use the CIFAR-10 (https://www.cs.toronto.edu/~kriz/cifar.html) for training and testing.

The notebook must present a confusion matrix and the average accuracy for each approach. You also have to report both training and test accuracies.



### Bag of Features ###

#### In order to enable the SITF module of the OpenCV library.<span style="color:red"> We install the opencv-contrib-python at version 3.4.2.16.</span> You should install it to make this notebook work. You can install it through any python package manager, since the module is in a pypi repository. More informations [here](https://pypi.org/project/opencv-contrib-python/3.4.2.16/) ####

#### Also you may need to install the <span style="color:red">scikit-image</span> module. Again it can be installed by your python package manager ####

In [1]:
!pip install opencv-contrib-python==3.4.2.16

     |████████████████████████████████| 30.6MB 1.2MB/s 
  Found existing installation: opencv-contrib-python 3.4.3.18
    Uninstalling opencv-contrib-python-3.4.3.18:
      Successfully uninstalled opencv-contrib-python-3.4.3.18


In [2]:
import numpy as np
import cv2 
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from keras.datasets import cifar10
from keras.utils import np_utils


##CARREGAR DATASET###
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

Using TensorFlow backend.


170508288/170498071 [==============================] - 2s 0us/step


In [0]:
from skimage import img_as_ubyte

def featureExtraction(sift,image):
    image = img_as_ubyte(image)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return keypoints, descriptors

In [4]:
sift = cv2.xfeatures2d.SIFT_create()
descriptors_list = []
for image in X_train:
    keypoints, descriptors = featureExtraction(sift,image)
    descriptors_list.append(descriptors)
    
print(len(descriptors_list))
print(descriptors_list[0].shape)

/usr/local/lib/python2.7/dist-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))


50000
(14, 128)


### Clustering

In [8]:
def build_histogram(descriptor_list, cluster_alg):
    histogram = np.zeros(len(cluster_alg.cluster_centers_))
    cluster_result =  cluster_alg.predict(descriptor_list)
    for i in cluster_result:
        histogram[i] += 1.0
    return histogram

print(descriptors_list[0].shape)
flatten_descriptors_list = []
for sublist in descriptors_list:
    if not sublist is None:  
      for item in sublist:
          flatten_descriptors_list.append(item)
print("Finished creating flatten descriptor vectors!")

k_values = [10]
for k in k_values:
  kmeans = KMeans(n_clusters = k)
  kmeans.fit(flatten_descriptors_list)
  image_bags = []
  for image_descriptors in descriptors_list:
    if(image_descriptors is not None):
      histogram = build_histogram(image_descriptors, kmeans)
      image_bags.append(histogram)
  print(len(image_bags))
  print(image_bags[0].shape)
  print(image_bags[1].shape)
  neighbor = NearestNeighbors(n_neighbors = 20)
  neighbor.fit(image_bags)
  dist, result = neighbor.kneighbors([image_bags[0]])
  print(dist)
  print(result)


    
    

(14, 128)
Finished creating flatten descriptor vectors!
49906
(10,)
(10,)
[[0.         1.         1.41421356 1.41421356 1.41421356 1.73205081
  1.73205081 1.73205081 1.73205081 1.73205081 1.73205081 1.73205081
  1.73205081 1.73205081 1.73205081 1.73205081 2.         2.
  2.         2.        ]]
[[    0 42667 21795 47612 23830 49188     3  8636  8694 21878 37978 35984
  38798 30039 41359 41639  4360 15749  8376 49318]]


In [0]:
print(descriptors_list[0].shape,descriptors_list[1].shape,descriptors_list[2].shape)

### ConvNet ###

In [0]:
# Plot ad hoc CIFAR10 instances
from keras.datasets import cifar10
from matplotlib import pyplot
from PIL import Image
# from scipy.misc import toimage -> DEPRECATED
from keras.layers import *

# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [0]:
# create a grid of 3x3 images
for i in range(0, 9):
	pyplot.subplot(330 + 1 + i)
	pyplot.imshow(Image.fromarray(X_train[i]))
# show the plot
pyplot.show()

#### Simple Covolutional Neural Network for CIFAR-10 ####

In [0]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.layers import *
K.set_image_dim_ordering('th')

seed = 7
numpy.random.seed(seed)

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


#### Larger Covolutional Neural Network for CIFAR-10 ####

In [0]:
# Create the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

numpy.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

#### Architecture Inspired in the LeNet5 ####

![lenet5](imgs/lenet5.jpg)

In [0]:
model = Sequential()

# model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), activation='relu', padding='same'))
model.add(Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(3,32,32)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(units=120, activation='relu'))
model.add(Dense(units=84, activation='relu'))
model.add(Dense(units=10, activation = 'softmax'))

epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

numpy.random.seed(seed)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))